In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *

from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals

In [2]:
def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print(full)
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    return False

#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

#generates random rank 4+ (hopefully) tensors
def rank_for():
    return  (low_tensor() + low_tensor() + low_tensor() + low_tensor())


# generates rank 1, 2x2x3 tensors
def low_tensor():
    max = 20
    a = np.random.randint(1, max, size=3)
    b = np.random.randint(1, max, size=4)
    c = np.random.randint(1, max, size=4)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 4, 4)) * 1.0
    return (tens,a,b,c)

In [3]:
def check(t, p = False):
    p000 = t[0][0][0]
    p001 = t[1][0][0]
    p002 = t[2][0][0]
    p100 = t[0][1][0]
    p101 = t[1][1][0]
    p102 = t[2][1][0]
    p010 = t[0][0][1]
    p011 = t[1][0][1]
    p012 = t[2][0][1]
    p110 = t[0][1][1]
    p111 = t[1][1][1]
    p112 = t[2][1][1]
    p210 = t[0][2][1]
    p211 = t[1][1][2]
    p212 = t[2][2][1]
    p220 = t[0][2][2]
    p221 = t[1][2][2]
    p222 = t[2][2][2]
    p120 = t[0][1][2]
    p121 = t[1][1][2]
    p122 = t[2][1][2]
    p200 = t[0][2][0]
    p201 = t[1][2][0]
    p202 = t[2][2][0]
    p020 = t[0][0][2]
    p021 = t[1][0][2]
    p022 = t[2][0][2]
    
def check_req(p):
    d1 = Matrix(p[0])
    d2 = Matrix(p[1])
    d3 = Matrix(p[2])
    v1 = Matrix(p[:,:,0])
    v2 = Matrix(p[:,:,1])
    v3 = Matrix(p[:,:,2])
    h1 = Matrix(p[:,0])
    h2 = Matrix(p[:,1])
    h3 = Matrix(p[:,2])
    zero = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    d1a = d1.adjugate()
    d2a = d2.adjugate()
    d3a = d3.adjugate()
    v1a = v1.adjugate()
    v2a = v2.adjugate()
    v3a = v3.adjugate()
    h1a = h1.adjugate()
    h2a = h2.adjugate()
    h3a = h3.adjugate()
    a1 = d1 * d2a * d3 - d3* d2a * d1
    a2 = d2 * d3a * d1 - d1* d3a * d2
    a3 = d3 * d1a * d2 - d2* d1a * d3
    b1 = v1 * v2a * v3 - v3* v2a * v1 
    b2 = v2 * v3a * v1 - v1* v3a * v2 
    b3 = v3 * v1a * v2 - v2* v1a * v3 
    c1 = h1 * h2a * h3 - h3* h2a * h1
    c2 = h2 * h3a * h1 - h1* h3a * h2 
    c3 = h3 * h1a * h2 - h2* h1a * h3 
    #print(a1,a2,a3)
    #print(a,b,c)
    a = check(a1) and check(a2) and check(a3)
    b = check(b1) and check(b2) and check(b3)
    c = check(c1) and check(c2) and check(c3)
    #if not (a and b and c):
    #  print(a,b,c)
    #  print(a1,b1,c1)
    return a or b or c
    
def check(a):
  #account for numerical errors
    return max(abs(a)) < 5

def check_for(p):
    d1 = Matrix(p[0])
    d2 = Matrix(p[1])
    d3 = Matrix(p[2])
    d2a = d2.adjugate()
    a1 = d1 * d2a * d3 - d3* d2a * d1
    #print(a1)
    return check(a1)

wrong = []

In [10]:
res = {True:0, False:0}

for i in range(1000000):
    t1 = low_tensor()
    t2 = low_tensor()
    t3 = low_tensor()
    t4 = low_tensor()
    t5 = low_tensor()
    v = np.random.randint(0,100)
    t = t1[0] + t2[0] + t3[0] + t4[0] + t5[0]
    if v%2 == 0:
        t = np.random.randint(0,100000,size = (3,3,3))
    key = check_for(t)
    res[key] = res[key] + 1
    if key:
        print(t1, t2, t3, t4, t5, t, v)
        print(check_rank(t,3, tol = 0.01))
        wrong.append((t1,t2,t3,t4,t5,t,v))
print(res)

(array([[[ 684., 1254., 1026., 1254.],
        [1368., 2508., 2052., 2508.],
        [1254., 2299., 1881., 2299.],
        [ 342.,  627.,  513.,  627.]],

       [[ 468.,  858.,  702.,  858.],
        [ 936., 1716., 1404., 1716.],
        [ 858., 1573., 1287., 1573.],
        [ 234.,  429.,  351.,  429.]],

       [[ 360.,  660.,  540.,  660.],
        [ 720., 1320., 1080., 1320.],
        [ 660., 1210.,  990., 1210.],
        [ 180.,  330.,  270.,  330.]]]), array([19, 13, 10]), array([ 6, 12, 11,  3]), array([ 6, 11,  9, 11])) (array([[[1260.,  560.,  560.,  560.],
        [1638.,  728.,  728.,  728.],
        [ 630.,  280.,  280.,  280.],
        [2016.,  896.,  896.,  896.]],

       [[1620.,  720.,  720.,  720.],
        [2106.,  936.,  936.,  936.],
        [ 810.,  360.,  360.,  360.],
        [2592., 1152., 1152., 1152.]],

       [[1440.,  640.,  640.,  640.],
        [1872.,  832.,  832.,  832.],
        [ 720.,  320.,  320.,  320.],
        [2304., 1024., 1024., 1024.]]]), a

In [11]:
t = [[[2965., 6039., 6544., 3713.],
  [4235. ,8448., 7720., 4940.],
  [2965., 6039., 6544., 3713.],
  [5510. ,7352., 4940., 2964.]],

 [[1944., 1896., 3213., 1465.],
  [2068., 2522., 2675., 1623.],
  [1944., 1896., 3213., 1465.],
  [1944., 2412., 1890., 1172.]],

 [[2748., 3302., 4621., 3650.],
  [3948., 5665., 5183., 4024.],
  [2748., 3302., 4621., 3650.],
  [3544., 4558., 3306., 2593.]]]

In [13]:
check_rank(tl.tensor(t),5,tol = 0.01, p = True)

0.0026656116678957687


True

In [21]:
P1 = Matrix(t[:,0] + t[:,1] + t[:,2]).transpose()
P2 = (t[:,:,0] + t[:,:,1] + t[:,:,2]).transpose()
P3 = (t[0] + t[1] + t[2]).transpose()
A = P2 * P1.adjugate() * P3
A[0,0]

2.19766299362767e+17

In [72]:
def cond4(t):
    P1 = Matrix(t[:,0] + t[:,1] + t[:,2]).transpose()
    P2 = (t[:,:,0] + t[:,:,1] + t[:,:,2]).transpose()
    P3 = (t[0] + t[1] + t[2]).transpose()
    if det(P1) == 0:
        print("found sth", t)
        return True
    A = det(P1) * P2 * P1.adjugate() * P3
    c1 = det(A) > 0
    c2 = A[0,0] > 0
    c3 = A.minor(2,2) > 0
    print(det(A), A[0,0], A.minor(2,2), det(P1))
    return c1 and c2 and c3

In [77]:
cond4(rank_for())

1.97197837166412e+84 8.68191250035974e+28 3.61879102864704e+57 1766093896020.00


True